In [1]:
import numpy as np
import gerrymetrics as g
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'svg'
import ipywidgets as wdgt

In [75]:
avgcolor = 'forestgreen'

def make_plot(avg=.57, comp_prop=.25, comp_range=.05, N=4000):
    N_comp = round(N*comp_prop)

    comp_votes_hi = np.random.rand(round(N_comp/2))*comp_range + avg
    comp_votes_lo = -comp_votes_hi + 2*avg

    other_votes_hi = np.random.rand(round((N-N_comp)/2))*(1-avg-comp_range) + avg + comp_range
    # other_votes_lo = np.random.rand(round((N-N_comp)/2))*(avg-comp_range)
    other_votes_lo = -other_votes_hi + 2*avg

    Dvotes = np.concatenate((comp_votes_hi, comp_votes_lo, other_votes_hi, other_votes_lo))

    fig, ax = g.plot_seats_votes_curve(Dvotes)
    fig.set_size_inches(8, 8)

    plt.scatter(.5, N/2, label='PB=0, MM=0')
    plt.axvline(avg, label='avg vote', color=avgcolor)
    ax.hist(Dvotes, bins=50, label='election histogram', color='mediumpurple')
    plt.legend()
    plt.ylim([0, N])
    plt.xlim([0, 1])


layout = wdgt.Layout(width='650px')
style = {'description_width': '400px'}

avg_widget = wdgt.FloatSlider(value=.57, min=0.0, max=1, step=0.01, description='average 2-party voteshare', continuous_update=False, style=style, layout=layout)
avg_widget.style.handle_color = avgcolor
N_widget = wdgt.IntSlider(value=4000, min=40, max=10000, step=10, description='# of districts', continuous_update=False, style=style, layout=layout)
comp_prop_widget = wdgt.FloatSlider(value=.25, min=0.0, max=1, step=0.01, description='proportion of districts that are competitive', continuous_update=False, style=style, layout=layout)
comp_range_widget = wdgt.FloatSlider(value=.05, min=0.0, max=0.4, step=0.01, description='competitiveness is defined as the average ± this:', continuous_update=False, style=style, layout=layout)

wdgt.HBox([wdgt.Label('A too long description'), comp_range_widget])


interactive_plot = wdgt.interactive(make_plot, avg=avg_widget, N=N_widget, comp_prop=comp_prop_widget, comp_range=comp_range_widget)
output = interactive_plot.children[-1]
output.layout.height = '700px'
interactive_plot

interactive(children=(FloatSlider(value=0.57, continuous_update=False, description='average 2-party voteshare'…